# Vježbe 4 - dio 2
 - Logistička regresija (nastavak s prošlih vježbi)
 - Unakrsna validacija
 - Linearna regresija kao klasifikacijski model
 - Matrica zabune
 - Višeklasna klasifikacija

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs, load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifier
from sklearn.metrics import confusion_matrix

from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

plt.rcParams["figure.figsize"] = (7, 7)

## Što je logistička regresija?
- za model funkciju koristimo logističku (sigmoidnu) funkciju $\sigma : \mathbb{R} \to \mathbb{R}$, zadanu s $\sigma(x) = \frac{1}{1+\exp{(-x)}}$
- model funkcija je oblika $h_{\Theta}(x) = \sigma(\Theta^Tx) = \frac{1}{1+\exp{(-\Theta^Tx)}}$, eventualno s dodatnim parametrom $\theta_0$.
- logistička funkcija preslikava cijeli $\mathbb{R}$ na interval $\langle 0,1 \rangle$ pa izlaznu vrijednost interpretiramo kao vjerojatnost da podatak $x$ s obzirom na parametar $\theta$ pripada klasi $1$
- kao u SVM ili perceptron modelu, $\Theta$ definira hiperravninu koja klasificira podatke, sve što je "ispod" je klasa 0, sve "iznad" klasa 1
- Učenje se svodi na minimizaciju **konveksne** funkcije cilja 
$$
J(\Theta) = \sum_{i=1}^m\log{}(1 + \exp(-y^{(i)}\Theta^Tx^{(i)})),
$$
što postižemo primjenom gradijentne metode ili nekog drugog optimizacijskog algoritma

In [ ]:
def lr_contour(X, y, model):
    plt.scatter(X[:, 0], X[:, 1], c=y)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 1000),
                         np.linspace(ylim[0], ylim[1], 1000))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, cmap='spring', alpha=0.2)
    plt.show()

In [ ]:
X = np.array([[-3.78,3.96],[-3.98,3.8],[-0.95,5.18],[-2.71,4],[-0.6,4.31],[-2.58,4.65],[-2.54,4.28],[-3.55,1.62],[-3.13,4.4],[-1.93,5.62]])
y = np.array([-1, -1, 1, 1, 1, 1, -1, -1, -1, 1])

lr = LogisticRegression().fit(X, y)
lr_contour(X, y, lr)

In [ ]:
a_df = pd.read_csv('./Podaci/A.csv')
X_a = a_df[['x1', 'x2']].to_numpy()
y_a = a_df['y'].to_numpy()
lr_a = LogisticRegression().fit(X_a, y_a)
lr_contour(X_a, y_a, lr_a)

## Linearna regresija kao klasifikacijski model
Neka su zadani podaci (u obliku matrice dizajna) $X\in \mathbb{R}^{m \times n}$ i njihove pripadne klase $y\in \{-1,1\}^m$. Pokušajmo riješiti problem učenjem modela linearne regresije.

Tražimo parametre $\Theta \in \mathbb{R}^n, \theta_0 \in \mathbb{R}$ takve da minimiziraju funkciju cilja 
$$
J(\Theta, \theta_0) = \sum\limits_{i=1}^m (h_{\Theta, \theta_0}(x^{(i)}) - y^{(i)})^2,$$
gdje je $h_{\Theta, \theta_0}(x) = \theta_0 + \theta_1 x_1 + \dots + \theta_n x_n$. 

Vrijednost model-funkcije koristimo za interpretaciju pripadnosti klase:
 - ako je $h_{\Theta, \theta_0}(x) > 0$, onda je $y = 1$,
 - ako je $h_{\Theta, \theta_0}(x) < 0$, onda je $y = -1$.

In [ ]:
X, y = make_blobs(n_samples=10000, n_features=2, centers=2, random_state=125)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=321)
y_train[y_train==0]=-1
y_test[y_test==0]=-1

lr = LinearRegression().fit(X_train, y_train)
log = LogisticRegression().fit(X_train, y_train)
rdg = RidgeClassifier().fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f'Accuracy modela linearne regresije = {accuracy}')

In [ ]:
y_pred = log.predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f'Accuracy modela logističke regresije = {accuracy}')

In [ ]:
y_pred = rdg.predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f'Accuracy modela ridge klasifikatora = {accuracy}')

In [ ]:
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 1000),
                     np.linspace(ylim[0], ylim[1], 1000))
Z = lr.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)


plt.contourf(xx, yy, Z, levels=[-100, 0, 100], cmap='spring', alpha=0.2)
plt.show()

## Matrica zabune
Pretpostavimo da je zadan problem određivanja zdravlja pacijenta na temelju nekakvih nalaza. Očito je da bolesne pacijente želimo zadržati na liječenju, a zdrave otpustiti. Postoje sljedeće četiri mogućnosti:
 - zdravom pacijentu (oznaka 0) smo predvidjeli da je zdrav (oznaka 0)
 - zdravom pacijentu (oznaka 0) smo predvidjeli da je bolestan (oznaka 1) - **greška 1. vrste**
 - bolesnom pacijentu (oznaka 1) smo predvidjeli da je bolestan (oznaka 1)
 - bolesnom pacijentu (oznaka 1) smo predvidjeli da je zdrav (oznaka 0) - **greška 2. vrste**

Ovisno o vrsti greške koju model napravi, moguće je da zdravog pacijenta zadržimo na dodatnom liječenju (što je donekle u redu), ili da bolesnog pacijenta otpustimo (što nikako nije u redu).

Ako želimo osigurati da bolesni pacijenti budu zadržani na liječenju, možemo konstruirati model koji će uvijek vraćati oznaku 1. Međutim, tada i sve zdrave pacijente zadržavamo. 

Za bolju evaluaciju rezultata uvodimo nove oznake i metrike. Pretpostavimo da se skup na kojem evaluiramo model sastoji od $m$ elemenata.
 - TP (true positive) $= |\{i : h(x^{(i)}) = y^{(i)} = 1\}|$, broj bolesnih pacijenata koje smo točno klasificirali
 - FP (false positive) $= |\{i : h(x^{(i)}) = 1, y^{(i)} = 0\}|$, broj zdravih pacijenata koje smo klasificirali kao bolesne
 - TN (true negative) $= |\{i : h(x^{(i)}) = y^{(i)} = 0\}|$, broj zdravih pacijenata koje smo točno klasificirali
 - FN (false negative) $= |\{i : h(x^{(i)}) = 0, y^{(i)} = 1\}|$, broj bolesnih pacijenata koje smo klasificirali kao zdrave
 
Definiramo metrike **precision** i **recall** kao
$$
\textrm{Precision} = \frac{TP}{TP + FP} \\
\textrm{Recall} = \frac{TP}{TP + FN}
$$

<center><img src="Precisionrecall.svg"/></center>

Grafički prikaz precision i recall metrika, preuzet s [Wikipedije](https://en.wikipedia.org/wiki/Precision_and_recall).

Rezultate možemo zapisati u tzv. **matricu zabune**, tablicu frekvencija gdje je svaki element frekvencija jedne od izračunatih klasa.

U [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) se nalazi funkcija koja računa matricu zabune.

Matrica zabune se također može generalizirati na problem klasifikacije s više klasa.

In [ ]:
y_true = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
y_pred = [0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0]
confusion_matrix(y_true, y_pred)

## Zadatak
Konstruirajte umjetan skup podataka koji nije linearno separabilan, podijelite ga na skup za treniranje i testiranje, naučite SVM model i ispišite matricu zabune na skupu za testiranje.

## Klasifikacija sa više klasa
Problem koji ćemo pokušati riješiti je klasifikacija znamenaka. Svaki podatak je slika na kojoj se nalazi jedna znamenka i pripadna znamenka.

### OvR
Za svaku od $k$ klasa konstruiramo binarni klasifikator, od kojih $i$-ti određuje je li dana znamenka jednaka $i$ ili različita od $i$.

Kao rezultat uzimamo onu klasu kojoj je pridružena najveća vjerojatnost (ili nekakav drugi skor).

In [ ]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5,figsize=(15,15))
for ax in axs:
    x = np.random.randint(9)
    ax.imshow(digits.data[x].reshape(8,8), cmap=plt.cm.binary)
    ax.set_xlabel('x-label', fontsize=12)
    ax.set_ylabel('y-label', fontsize=12)
    ax.set_title(digits.target[x], fontsize=14)
    ax.axis('off')

In [ ]:
def train(X, y, digit):
    clf = SVC(probability=True, kernel='linear', gamma='auto')
    y = np.array([1 if i == digit else -1 for i in y])
    clf.fit(X, y)
    return clf

def predict(X, models):
    predictions = np.zeros((X.shape[0], 10))
    for i, model in enumerate(models):
        pred = model.predict_proba(X)[:, 1]
        predictions[:, i] = pred
    return np.argmax(predictions, axis=1)

In [ ]:
models = []
for digit in range(10):
    models.append(train(X_train, y_train, digit))

y_pred = predict(X_test, models)
np.sum(y_pred == y_test) / len(y_test)

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3,figsize=(15,15))
for i, ax in enumerate(axs.flat):
    ax.imshow(X_test[i].reshape(8,8), cmap=plt.cm.binary)
    ax.set_title(f'y = {y_test[i]}, y_pred = {y_pred[i]}')
    ax.axis('off')

Na isti način možemo iskorisiti model logističke regresije.

### OvO
Drugi način klasifikacije je treniranje $\frac{k(k-1)}{2}$ modela od kojih svaki računa skorove za $i$-tu, odnosno $j$-tu klasu.

Možete naslutiti da je treniranje OvO sporije od OvR jer je potrebno naučiti $O(k^2)$ različitih modela. Ipak, za učenje modela $i,j$ koristi se podskup skupa za treniranje gdje su svi podaci klase $i$ ili $j$, što donekle ubrzava učenje.

In [ ]:
%%timeit
ovo = OneVsOneClassifier(SVC(random_state=123)).fit(X_train, y_train)

In [ ]:
%%timeit
ovr = OneVsRestClassifier(SVC(random_state=123)).fit(X_train, y_train)

### Ugrađena višeklasna klasifikacija
U ugrađenom SVM solveru su dostupne OvO i OvR strategije.

U modelu logističke regresije dostupni su OvO i multinomna strategija.

In [ ]:
X, y = make_blobs(n_samples=10000, n_features=2, centers=3, random_state=123)
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = SVC().fit(X_train, y_train)
np.sum(y_test == clf.predict(X_test)) / len(y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = LogisticRegression(multi_class='multinomial').fit(X_train, y_train)
np.sum(y_test == clf.predict(X_test)) / len(y_test)